# 1. Create .csv file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from pandas import DataFrame

### Load Data

In [ ]:
data_file="/content/drive/MyDrive/CMMD/GNN/10mm"
file_list=os.listdir(data_file)
len(file_list)
file_list

['0725', '0725_make_csv(rpt_v2).ipynb']

In [ ]:
format = []
for file in  file_list:
  if file.endswith(".inp"):
        design_name=file.rpartition('.')[0]
        format.append(design_name)

format

## 

### U

In [ ]:
def rpt_start(rpt):
    f = open(rpt, "r")
    i=1
    while True:
        #print(i)
        line = f.readline()
        #print(line)
        #"*Element"
        #"Nset"
        if not line:
            break
        #if line.startswith("---------------------------------------------------------------------------------"):
        if line.startswith("  Part Instance     Node ID                   Orig. Coords                                     Def. Coords"):
            print('~~~~~~~~~~~~~ u : START ~~~~~~~~~~~~~~~~~~~~~~~~')
            print(i+2)
            #print(line)
            #start = i
            start = i+2
        i = i+1

    print('start: {0}'.format(start))

    return start

def rpt_end(rpt, rpt_start_line):
    f = open(rpt, "r")
    i=1
    while True:
        #print(i)
        line = f.readline()
        #"*Element"
        #"Nset"
        if not line:
            break
        if (line.startswith("  Part Instance     Node ID  Attached elements    U, Magnitude") and (i > rpt_start_line)):
            print('~~~~~~~~~~~~~ u : end ~~~~~~~~~~~~~~~~~~~~~~~~')
            print("end")
            print(i-3)
            end = i-3
            break
        i = i+1
    print('end: {0}'.format(end))

    return end

In [ ]:
def start_end_rpt(rpt):
    # 필요한 부분
  f = open(rpt, "r")
  i=1
  while True:
      #print(i)
      line = f.readline()
      #"*Element"
      #"Nset"
      if not line:
          break
      if line.startswith("  Part Instance     Node ID                   Orig. Coords                                     Def. Coords"):
        print('~~~~~~~~~~~~~ uu : START ~~~~~~~~~~~~~~~~~~~~~~~~')
        print("start_line")
        print(i)
        start = i

      if line.startswith("  Part Instance     Node ID  Attached elements    U, Magnitude"):
        print('~~~~~~~~~~~~~ u : END ~~~~~~~~~~~~~~~~~~~~~~~~')
        print("Finish_line")
        print(i-3)
        end = i-2
        break
      i = i+1

  print('start: {0}, end: {1}'.format(start, end ))

  return start,end



def node_deform(rpt):

  #start,end = start_end_rpt(rpt)

  rpt_start_line = rpt_start(rpt)
  print(rpt_start_line)
  rpt_end_line = rpt_end(rpt,rpt_start_line)
  print(rpt_end_line)
  print('-----------deformation---------------')



  f = open(rpt, "r")
  i=1

  # U_DF 생성
  U_DF = DataFrame(columns=['node_id', 'U', 'U1', 'U2', 'U3'])
  while True:
      line = f.readline()
      if not line:
          break
      if (rpt_start_line < i <rpt_end_line):
        #print(line)

        elements = line.split(' ')
        elements = list(filter(None, elements))

        #elements = line.split(',')
        #elements = list(map(lambda s : s.strip(), elements))
        x = float(elements[2])
        y = float(elements[3])
        z = float(elements[4])
        U1 = float(elements[5]) - float(elements[2])
        U2 = float(elements[6]) - float(elements[3])
        U3 = float(elements[7]) - float(elements[4])
        U = (U1**2 + U2**2 + U3**2)**0.5


        # new_DF = DataFrame([elements[1], elements[1], elements[2] , elements[3]  , elements[4]]).transpose()
        new_DF = pd.DataFrame({'node_id': [elements[1]], 'x': [x], 'y': [y], 'z':[z], 'U': [U], 'U1': [U1], 'U2': [U2], 'U3': [U3]})
        U_DF = pd.concat([U_DF, new_DF], ignore_index=True)

        #print(elements)
        '''
        for num , element in enumerate(elements):
          if (0 < num < 5):
            a.append(element)
          #print(element)
          '''

      i +=1


  return U_DF



###  x y z

### element + real_node

In [ ]:
def start_inp_element(inp):
  f = open(inp, "r")
  i=1
  while True:
    #print(i)
    line = f.readline()
    #"*Element"
    #"Nset"
    if not line:
        break
    if line.startswith("*Element, type=C3D10"):
      print("Element")
      print(i)
      start_element = i+1
      break
    i = i+1

  print('start')
  print(start_element)
  return start_element

def end_inp_element(inp, start_element):
  f = open(inp, "r")
  i=1
  while True:
    line = f.readline()
    if not line:
        break
    if line.startswith("*Nset"):
      if i > start_element:
        print("Nset")
        print(i)
        end_element = i
        #print(line)
        break
    i = i+1

  print('end')
  print(end_element)
  return end_element

In [ ]:
def element(inp):

  start_element = start_inp_element(inp)
  end_element = end_inp_element(inp, start_element)
  print('-----------element---------------')

  f = open(inp, "r")
  i=1
  # create U_DF 
  node_list = []
  elementsDF = DataFrame(columns=['element_id', 'node1', 'node2', 'node3', 'node4'])


  while True:
    line = f.readline()
    if not line:
        break
    ###### element 

    if (start_element-1 < i < end_element):
      #print(line)
      elements = line.split(',')
      elements = list(map(lambda s : s.strip(), elements))

      new_DF = DataFrame([elements[0], elements[1], elements[2] , elements[3]  , elements[4]]).transpose()
      new_DF = new_DF.rename(columns = {0:'element_id', 1:'node1', 2:'node2', 3:'node3', 4:'node4'})
      elementsDF = pd.concat([elementsDF, new_DF])

      #print(elements)
      for num , element in enumerate(elements):
        if (0 < num < 5):
          node_list.append(element)
        #print(element)

    i +=1



  return node_list, elementsDF

### surf df

In [ ]:

def surf(txt):
  f = open(txt,'r')
  surf_txt_real = f.read().split(',')

  elements = list(map(lambda s : s.strip(), surf_txt_real))

  surf_df_real = DataFrame(elements)

  surf_df_real = surf_df_real.drop(surf_df_real.index[-1])
  surf_df_real.rename(columns={0:'node_id'},inplace = True)
  surf_df_real = surf_df_real.astype(int)
  surf_df_real['node_type'] = 0

  return surf_df_real

### Processing functions

In [ ]:
for design_name in format:
  print('____________________________')
  print(design_name)
  print()
  inp = data_file+'/'+design_name+'.inp'
  rpt = data_file+'/'+design_name+'_v2.rpt'
  surf_txt = data_file+'/'+design_name+'_outer_node.txt'

  # U_DF = node_deform(rpt)
  # U_coordinate = node_xyz(inp)
  # node_final = pd.merge(U_coordinate, U_DF, on='node_id', how='right')
  node_final = node_deform(rpt)
  node_list =  element(inp)[0]
  elementsDF =  element(inp)[1]


  filtered_node_csv = node_final[node_final['node_id'].isin(node_list)]
  filtered_node_csv = filtered_node_csv.astype({'node_id': 'int64'})
  for col in filtered_node_csv.columns[1:]:
    filtered_node_csv = filtered_node_csv.astype({col: 'float'})



  surf_df_real = surf(surf_txt)

  node_list2 = set(node_list) # element csv 에서 찐 node만 가져온 건데 중복 제거
  node_list2 = list(node_list2) # 리스트 형식으로 변환
  node_list2=[item.rstrip(',') for item in node_list2]

  node_list3 = list(map(int, node_list2)) # string -> int 로 변환
  surf_df_real_final = surf_df_real[surf_df_real['node_id'].isin(node_list3)]





  node_final_final = pd.merge(filtered_node_csv, surf_df_real_final, on='node_id', how='outer')
  node_final_final['node_type'] = node_final_final['node_type'].fillna(1)



  node_final_surf = pd.merge(filtered_node_csv, surf_df_real_final, on='node_id', how='right')
  #node_final_final['node_type'] = node_final_final['node_type'].fillna(1)

  node_final_final['node_id'] = node_final_final['node_id'] - 1 # node id starts with 0
  node_final_surf['node_id'] = node_final_surf['node_id'] - 1 # node id starts with 0



  nodes_csv_path = '/content/drive/MyDrive/BMML/10mm_rpt2/csv_rpt2/' + design_name + '_nodes.csv'
  with open(nodes_csv_path, 'w', encoding = 'utf-8-sig') as f:
    node_final_final.to_csv(f)

  nodes_surf_csv_path = '/content/drive/MyDrive/BMML/10mm_rpt2/csv_rpt2/' + design_name + '_surfnodes.csv'
  with open(nodes_surf_csv_path, 'w', encoding = 'utf-8-sig') as f:
    node_final_surf.to_csv(f)

  elementsDF = elementsDF[elementsDF['node1'].isin(node_list)]
  elementsDF = elementsDF[elementsDF['node2'].isin(node_list)]
  elementsDF = elementsDF[elementsDF['node3'].isin(node_list)]
  elementsDF = elementsDF[elementsDF['node4'].isin(node_list)]

  elementsDF = elementsDF.astype('int64')
  for col in elementsDF.columns:
    elementsDF[col] = elementsDF[col]-1

  elements_csv_path = '/content/drive/MyDrive/BMML/10mm_rpt2/csv_rpt2/' + design_name+'_elements.csv'
  with open(elements_csv_path, 'w', encoding = 'utf-8-sig') as f:
    elementsDF.to_csv(f)

  f_txt = open(surf_txt,'r')
  surf_txt_real = f_txt.read().split(',')
  surf_node = list(map(lambda s : s.strip(), surf_txt_real))
  surf_node2 = set(surf_node) #
  surf_node3 = list(surf_node2) # 
  surf_node4 = list(filter(None, surf_node3))
  surf_node5 = list(map(int, surf_node4)) # string -> int 

  surf_elementsDF = elementsDF[elementsDF['node1'].isin(surf_node5)]
  surf_elementsDF = surf_elementsDF[surf_elementsDF['node2'].isin(surf_node5)]
  surf_elementsDF = surf_elementsDF[surf_elementsDF['node3'].isin(surf_node5)]
  surf_elementsDF = surf_elementsDF[surf_elementsDF['node4'].isin(surf_node5)]


  elements_surf_csv_path = '/content/drive/MyDrive/BMML/10mm_rpt2/csv_rpt2/' + design_name+'_surfelements.csv'
  with open(elements_surf_csv_path, 'w', encoding = 'utf-8-sig') as f:
    surf_elementsDF.to_csv(f)


**1. elements.csv**

[element_id, node1, node2, node3, node4]    

(5966,5): elementsDF

**2. nodes.csv**

[node_id, x, y, z, U, U1, U2, U3, node_tyoe]

(1384,9):node_final_final

**3. surfelements.csv**

[element_id, node1, node2, node3, node4]

(61,5): surf_elementsDF

**4. surfnodes.csv**

[node_id, x, y, z, U, U1, U2, U3, node_tyoe]

(716,9): node_final_surf

## final DF.csv 

In [ ]:
elementsDF

In [ ]:
node_final_final

In [ ]:
surf_elementsDF

In [ ]:
node_final_surf